In [ ]:
# !pwd

In [1]:
# %cd pstage4gun

/opt/ml/pstage4gun


In [2]:
import argparse
from datetime import datetime
import os
import yaml
from typing import Any, Dict, Tuple, Union

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder, VisionDataset
from torch.utils.data import DataLoader, random_split

from src.dataloader import create_dataloader
from src.loss import CustomCriterion
from src.model import Model
from src.trainer import TorchTrainer
from src.utils.common import get_label_counts, read_yaml
from src.utils.macs import calc_macs
from src.utils.torch_utils import check_runtime, model_info

CLASSES = ['Battery', 'Clothing', 'Glass', 'Metal', 'Paper', 'Paperpack', 'Plastic', 'Plasticbag', 'Styrofoam']

model_config = 'configs/model/mobilenetv3.yaml'
weight = 'exp/2021-05-25_14-36-26/best.pt'
img_root_val = '/opt/ml/input/data/val/'
img_root_test = '/opt/ml/input/data/test/'
data_config = 'configs/data/taco.yaml'

model_config = read_yaml(cfg=model_config)
data_config = read_yaml(cfg=data_config)

In [3]:
def val_inference(
    model_config: Dict[str, Any],
    data_config: Dict[str, Any],
    model_path: str,
    fp16: bool,
    device: torch.device,
) -> Tuple[float, float, float]:

    transform_test_params = data_config.get("AUG_TEST_PARAMS")
    if not transform_test_params:
        transform_test_params = dict()

    transform_test = getattr(
        __import__("src.augmentation.policies", fromlist=[""]),
        data_config["AUG_TEST"],
    )(dataset=data_config["DATASET"], img_size=data_config["IMG_SIZE"], **transform_test_params)
    val_dataset = ImageFolder(root=os.path.join(data_config['DATA_PATH'], 'val'),
                              transform=transform_test)

    valid_loader = DataLoader(
        dataset=val_dataset,
        pin_memory=(torch.cuda.is_available()),
        shuffle=False,
        batch_size=data_config["BATCH_SIZE"],
        num_workers=5
    )

    model_instance = Model(model_config, verbose=True)
    model_instance.model.to(device)

    # Calc macs
    macs = calc_macs(model_instance.model, (3, data_config["IMG_SIZE"], data_config["IMG_SIZE"]))
    print(f"macs: {macs}")

    # Create optimizer, scheduler, criterion
    criterion = CustomCriterion(
        samples_per_cls=get_label_counts(data_config["DATA_PATH"])
        if data_config["DATASET"] == "TACO"
        else None,
        device=device,
    )

    trainer = TorchTrainer(
        model=model_instance.model,
        criterion=criterion,
        optimizer=None,
        scheduler=None,
        scaler=None,
        device=device,
        model_path=model_path,
        verbose=1,
    )

    # evaluate model with test set
    model_instance.model.load_state_dict(torch.load(model_path))
    val_loss, val_f1, val_acc = trainer.test(
        model=model_instance.model, test_dataloader=valid_loader
    )
    return val_loss, val_f1, val_acc, macs

In [4]:
val_loss, val_f1, val_acc, model_macs = val_inference(
    model_config=model_config,
    data_config=data_config,
    model_path=weight,
    fp16=True,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

/miniconda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
 Val:       Loss: 9.237, Acc: 28.39% F1(macro): 0.12: 100%|██████████| 128/128 [00:53<00:00,  2.38it/s]


idx |   n |     params |          module |            arguments |   in_channel |   out_channel
----------------------------------------------------------------------------------------------
  0 |   1 |        464 |            Conv | [16, 3, 2, None, 1, 'HardSwish'] |            3           16
  1 |   1 |        464 | InvertedResidualv3 |  [3, 1, 16, 0, 0, 1] |           16           16
  2 |   1 |      3,440 | InvertedResidualv3 |  [3, 4, 24, 0, 0, 2] |           16           24
  3 |   1 |      4,440 | InvertedResidualv3 |  [3, 3, 24, 0, 0, 1] |           24           24
  4 |   1 |     10,328 | InvertedResidualv3 |  [5, 3, 40, 1, 0, 2] |           24           40
  5 |   1 |     20,992 | InvertedResidualv3 |  [5, 3, 40, 1, 0, 1] |           40           40
  6 |   1 |     20,992 | InvertedResidualv3 |  [5, 3, 40, 1, 0, 1] |           40           40
  7 |   1 |     32,080 | InvertedResidualv3 |  [3, 6, 80, 0, 1, 2] |           40           80
  8 |   1 |     34,760 | InvertedResidual

In [7]:
crit_MACs = 13863553
crit_F1 = 0.8342

score_max = model_macs / crit_MACs
if val_f1 < crit_F1:
    score_f1 = 1 - (val_f1 / crit_F1)
else:
    score_f1 = 0.5 * (1 - (val_f1 / crit_F1))

score = score_max + score_f1

print(f"val score: {score:.4f}, val f1: {val_f1:.4f}, MACs: {model_macs:.4f}")

val score: 17.0277, val f1: 0.1160, MACs: 224128545.0000
